# Understanding Retrieval Question Answering

### Setup

In [ ]:
pip install -q --upgrade google-generativeai langchain-google-genai

In [ ]:
pip install -qU langchain chromadb langchain-community "unstructured[md]"

In [1]:
import google.generativeai as genai
from getpass import getpass

In [2]:
import weave
weave.init("gemini-weave3_retrieval")

weave version 0.51.12 is available!  To upgrade, please run:
 $ pip install weave --upgrade
Logged in as Weights & Biases user: supriyagdptl.
View Weave data at https://wandb.ai/supriyagdptl/gemini-weave3_retrieval/weave


In [3]:
GOOGLE_GEMINI_API_KEY = getpass("Paste your Google Gemini API key from: https://aistudio.google.com/app/apikey\n")

Paste your Google Gemini API key from: https://aistudio.google.com/app/apikey
 ········


In [4]:
genai.configure(api_key=GOOGLE_GEMINI_API_KEY)

In [5]:
model_name = "models/gemini-1.5-pro"
model_info = genai.get_model(model_name)
print(model_info)

Model(name='models/gemini-1.5-pro',
      base_model_id='',
      version='001',
      display_name='Gemini 1.5 Pro',
      description='Mid-size multimodal model that supports up to 2 million tokens',
      input_token_limit=2000000,
      output_token_limit=8192,
      supported_generation_methods=['generateContent', 'countTokens'],
      temperature=1.0,
      max_temperature=2.0,
      top_p=0.95,
      top_k=64)


## Langchain

[LangChain](https://docs.langchain.com/docs/) is a framework for developing applications powered by language models. We will use some of its features in the code below. Let's start by configuring W&B tracing. 

In [6]:
import os

In [7]:
# we need a single line of code to start tracing langchain with W&B
os.environ["LANGCHAIN_WANDB_TRACING"] = "true"

# wandb documentation to configure wandb using env variables
# https://docs.wandb.ai/guides/track/advanced/environment-variables
# here we are configuring the wandb project name
os.environ["WANDB_PROJECT"] = "llmapps"

## Parsing documents

We will use a small sample of markdown documents in this notebook. Let's find them and make sure we can stuff them into the prompt. That means they may need to be chunked and not exceed some number of tokens. 

In [8]:
from langchain.document_loaders import DirectoryLoader

def find_md_files(directory):
    "Find all markdown files in a directory and return a LangChain Document"
    dl = DirectoryLoader(directory, "**/*.md")
    return dl.load()

documents = find_md_files('docs_sample/')
len(documents)

11

In [10]:
from langchain.text_splitter import MarkdownTextSplitter

md_text_splitter = MarkdownTextSplitter(chunk_size=1000)
document_sections = md_text_splitter.split_documents(documents)
len(document_sections)

103

let's look at the first section

In [22]:
document_sections[0].page_content

Document(metadata={'source': 'docs_sample/lightning.md'}, page_content="import Tabs from '@theme/Tabs';\n\nimport TabItem from '@theme/TabItem';\n\n# PyTorch Lightning\n\n[! [Open In Colab](https://colab.research.google.com/assets/colab\n\nbadge.svg)](https://wandb.me/lightning)\n\nPyTorch Lightning provides a lightweight wrapper for organizing your PyTorch code and easily adding advanced features such as distributed training and 16-bit precision. W&B provides a lightweight wrapper for logging your ML experiments. But you don't need to combine the two yourself: Weights & Biases is incorporated directly into the PyTorch Lightning library via the [**`WandbLogger`**](https://pytorch-lightning.readthedocs.io/en/stable/extensions/generated/pytorch\\_lightning.loggers.WandbLogger.html#pytorch\\_lightning.loggers.WandbLogger).\n\n## ⚡ Get going lightning-fast with just two lines.\n\n```python\n\nfrom pytorch_lightning.loggers import WandbLogger\n\nfrom pytorch_lightning import Trainer\n\nwand

## Embeddings

Let's now use embeddings with a vector database retriever to find relevant documents for a query. 


[Reference](https://github.com/google/generative-ai-docs/blob/main/examples/gemini/python/vectordb_with_chroma/vectordb_with_chroma.ipynb)

In [18]:
from IPython.display import Markdown
import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings

In [14]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [25]:
class GeminiEmbeddingFunction(EmbeddingFunction):
  def __call__(self, input: Documents) -> Embeddings:
    model = 'models/embedding-001'
    title = "Custom query"
    return genai.embed_content(model=model,
                                content=input,
                                task_type="retrieval_document",
                                title=title)["embedding"]


In [28]:
docs = [sections.page_content for sections in document_sections]

In [32]:
chroma_client = chromadb.Client()
db = chroma_client.get_or_create_collection(name="wandb", embedding_function=GeminiEmbeddingFunction())

for i, d in enumerate(docs):
    db.add(
      documents=d,
      ids=str(i))

We can create a retriever from the db now, we can pass the `k` param to get the most relevant sections from the similarity search

In [38]:
query = "How can I share my W&B report with my team members in a public W&B project?"
db.query(query_texts=[query], n_results=3)

{'ids': [['25', '26', '93']],
 'distances': [[0.27467256784439087, 0.3961881697177887, 0.4060346782207489]],
 'metadatas': [[None, None, None]],
 'embeddings': None,
 'documents': [["description: Collaborate and share W&B Reports with peers, co-workers, and your team.\n\nCollaborate on reports\n\nOnce you have saved a report, you can select the Share button to collaborate. A draft copy of the report is created when you select the Edit button. Draft reports auto-save. Select Save to report to publish your changes to the shared report.\n\nA warning notification will appear if an edit conflict occurs. This can occur if you and another collaborator edit the same report at the same time. The warning notification will guide you to resolve potential edit conflicts.\n\nComment on reports\n\nClick the comment button on a panel in a report to add a comment directly to that panel.\n\nWho can edit and share reports?\n\nReports that are created within an individual's private project is only visible

In [44]:
relevant_docs = db.query(query_texts=[query], n_results=3)["documents"][0]

In [45]:
relevant_docs

["description: Collaborate and share W&B Reports with peers, co-workers, and your team.\n\nCollaborate on reports\n\nOnce you have saved a report, you can select the Share button to collaborate. A draft copy of the report is created when you select the Edit button. Draft reports auto-save. Select Save to report to publish your changes to the shared report.\n\nA warning notification will appear if an edit conflict occurs. This can occur if you and another collaborator edit the same report at the same time. The warning notification will guide you to resolve potential edit conflicts.\n\nComment on reports\n\nClick the comment button on a panel in a report to add a comment directly to that panel.\n\nWho can edit and share reports?\n\nReports that are created within an individual's private project is only visible to that user. The user can share their project to a team or to the public.",
 "Who can edit and share reports?\n\nReports that are created within an individual's private project is

## Stuff Prompt

We'll now take the content of the retrieved documents, stuff them into prompt template along with the query, and pass into an LLM to obtain the answer. 

In [46]:
from langchain.prompts import PromptTemplate

prompt_template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

context = "\n\n".join([doc for doc in relevant_docs])
prompt = PROMPT.format(context=context, question=query)

Use langchain to call openai chat API with the question

In [48]:
GOOGLE_GEMINI_API_KEY = getpass("Paste your Google Gemini API key from: https://aistudio.google.com/app/apikey\n")

Paste your Google Gemini API key from: https://aistudio.google.com/app/apikey
 ········


In [50]:
genai.configure(api_key=GOOGLE_GEMINI_API_KEY)

In [57]:
from langchain_google_genai import GoogleGenerativeAI
import os
os.environ["GOOGLE_API_KEY"] = GOOGLE_GEMINI_API_KEY
llm = GoogleGenerativeAI(model="models/gemini-1.5-pro")
response = llm.predict(prompt)
Markdown(response)

This content doesn't explicitly state whether reports within public projects can be shared with specific team members. It mainly describes sharing reports via email or magic links and managing report access within team projects. 


## Using Langchain

Langchain gives us tools to do this efficiently in few lines of code. Let's do the same using `RetrievalQA` chain.

In [60]:
db.as_retriever

AttributeError: 'Collection' object has no attribute 'as_retriever'

In [ ]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=GoogleGenerativeAI(model="models/gemini-1.5-pro"), 
                                 chain_type="stuff")
result = qa.run(query)

Markdown(result)

In [ ]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever)
result = qa.run(query)

Markdown(result)